# API

In [224]:
import requests
import json
import pandas as pd

#Api sobre recetas de comida
#https://www.themealdb.com/api.php

url = 'https://www.themealdb.com/api/json/v1/1/lookup.php?i=52'

#Añadimos a la lista recetas todas las recetas de la api
#La id de todas las recetas está entre 52100 y 52999
recetas = []
for i in range(100, 1000): 
    path = url + str(i)
    r = requests.get(path)
    recetas.append(r.text)

#Elimino de la lista todas las recetas cuya id sea nula
recetas = [x for x in recetas if x != '{"meals":null}']

#Hago un bucle para convertir al formato json las recetas y las añadimos a otra lista
nombre_rec = []
for c in recetas: 
    jsn = json.loads(c)
    nombre_rec.append(jsn['meals'])

nombre_rec = [e for x in nombre_rec for e in x]

#Rellenamos todos los valores nulos de cada columna
df = pd.DataFrame(nombre_rec, columns=['idMeal', 'strMeal', 'strCategory', 'strArea', 'strInstructions', 'strMealThumb', 'strTags', 'strYoutube', 'strIngredient1', 'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 'strIngredient10', 'strIngredient11', 'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15', 'strIngredient16', 'strIngredient17', 'strIngredient18', 'strIngredient19', 'strIngredient20', 'strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10', 'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15', 'strMeasure16', 'strMeasure17', 'strMeasure18', 'strMeasure19', 'strMeasure20', 'strSource', 'dateModified'])
df = df.fillna("")

#Fusiono todas las columnas de ingredientes con su respectiva medida
for x in range(1,21):
    df['Ingredients{}'.format(x)] = df[['strMeasure{}'.format(x), 'strIngredient{}'.format(x)]].apply(lambda x: ' '.join(x), axis=1)
for x in range(1,21):
    df = df.drop(['strIngredient{}'.format(x),'strMeasure{}'.format(x),], axis=1)

#Fusiono las listas de cada ingrediente en una sola lista con todos los ingredientes
df['Ingredients'] = df[['Ingredients1','Ingredients2','Ingredients3','Ingredients4','Ingredients5','Ingredients6','Ingredients7','Ingredients8','Ingredients9','Ingredients10','Ingredients11','Ingredients12','Ingredients13','Ingredients14','Ingredients15','Ingredients16','Ingredients17','Ingredients18','Ingredients19','Ingredients20']].apply(lambda x: ', '.join(x) , axis=1)
for x in range(1,21):
    df = df.drop(['Ingredients{}'.format(x)], axis=1)
    
#Elimino algunas columnas que me son irrelevantes
df = df.drop(['dateModified','idMeal','strTags','strYoutube'], axis=1)

#Renombro las columnas
df = df.rename(index=str, columns={'strMeal': "Meal", 'strCategory': "Category", "strArea": "Area", "strInstructions": "Instructions", 'strMealThumb':'MealThumb', 'strSource':'Source'})

# Exporto el dataframe para no tenerlo que volver a cargar de la api
data = df.to_csv('recipesapi.csv', index=False)

# Web Scraping

In [257]:
from bs4 import BeautifulSoup
import requests, time

lista = []

#Creo una función que itera dentro de las paginas de recetas de la web
#Que imprima el número de página y llame a la siguiente función que itera dentro de cada página
#Escogemos las recetas asiaticas
def all_recipes_asian(pages):
    for x in range(pages):
        url = "https://www.allrecipes.com/recipes/227/world-cuisine/asian/?page={}".format(x+1)
        dic = 'dic{}'.format(x)
        area = "Asian"
        print(x)
        call_recipe(url, dic, area)

#Creo una función que busca la url de cada receta en cada página de recetas de la función anterior
def call_recipe(url, dic, area):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html)
    urls = [element["href"] for element in soup.find_all("a",{'class':"fixed-recipe-card__title-link"})]
    for url in urls:
        #Imprime la url de la receta
        print(url)
        get_recipe(url, dic, area)

#Esta función agrega a una lista cada receta
def get_recipe(url, dic, area):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html)
    dic = {}
    time.sleep(1)
    
    for element in soup.find_all("h1",{'class':"recipe-summary__h1"}):
        dic['Meal'] = element.text.strip()
        
    for element in soup.find_all("span",{'class':'toggle-similar__title'}):
        dic['Category'] = element.text.replace("\n", "").strip()
    
    dic['Area'] = area

    sins = ""
    for element in soup.find_all("span",{'class':'recipe-directions__list--item'}):
        sins += element.text.replace("\n", "").strip() + "; "
    dic['Instructions'] = sins

    for element in soup.find_all("img",{'class':'rec-photo'}):
        dic['MealThumb'] = element['src']

    dic['Source'] = url

    sing = ""
    for element in soup.find_all("span",{'itemprop':'recipeIngredient'}):
        sing += element.text.replace("\n", "").strip() + "; "
    dic['Ingredients'] = sing
    return lista.append(dic)

#Llamamos a la primera función para buscar recetas
all_recipes_asian(50)
dfasian = pd.DataFrame(lista, columns=['Meal','Category','Area','Instructions','MealThumb','Source','Ingredients'])
dfasian

#Guardamos en un csv las recetas
data = dfasian.to_csv('recipesweb.csv', index=False)

0
https://www.allrecipes.com/recipe/14565/naan/
https://www.allrecipes.com/recipe/189477/california-roll-sushi/
https://www.allrecipes.com/recipe/246250/kima/
https://www.allrecipes.com/recipe/245993/soba-noodle-salad-with-chicken-and-sesame/
https://www.allrecipes.com/recipe/241806/spicy-baked-tofu/
https://www.allrecipes.com/recipe/239818/iranian-persian-salad-shirazi/
https://www.allrecipes.com/recipe/238977/lamb-korma/
https://www.allrecipes.com/recipe/230330/nuoc-cham-vietnamese-spicy-dipping-sauce/
https://www.allrecipes.com/recipe/228240/bibimbap-korean-rice-with-mixed-vegetables/
https://www.allrecipes.com/recipe/68532/curried-coconut-chicken/
https://www.allrecipes.com/recipe/86230/szechwan-shrimp/
https://www.allrecipes.com/recipe/45736/chicken-tikka-masala/
https://www.allrecipes.com/recipe/71722/asian-lettuce-wraps/
https://www.allrecipes.com/recipe/86687/broccoli-with-garlic-butter-and-cashews/
https://www.allrecipes.com/recipe/16954/chinese-chicken-fried-rice-ii/
https://

https://www.allrecipes.com/recipe/14624/rib-eye-steaks-with-a-soy-and-ginger-marinade/
6
https://www.allrecipes.com/recipe/28363/adobo-chicken-with-ginger/
https://www.allrecipes.com/recipe/18517/napa-cabbage-salad/
https://www.allrecipes.com/recipe/15177/famous-japanese-restaurant-style-salad-dressing/
https://www.allrecipes.com/recipe/83117/asian-pork-tenderloin/
https://www.allrecipes.com/recipe/158854/chinese-buffet-green-beans/
https://www.allrecipes.com/recipe/185519/homemade-wonton-soup/
https://www.allrecipes.com/recipe/86828/grilled-chicken-thighs-tandoori/
https://www.allrecipes.com/recipe/46819/chicken-broccoli-ca-uniengs-style/
https://www.allrecipes.com/recipe/68813/spicy-basil-chicken/
https://www.allrecipes.com/recipe/241601/sesame-chicken-for-slow-cooker/
https://www.allrecipes.com/recipe/64678/fast-and-easy-tofu-lo-mein/
https://www.allrecipes.com/recipe/14759/pork-dumplings/
https://www.allrecipes.com/recipe/39380/broccoli-beef-ii/
https://www.allrecipes.com/recipe/72

https://www.allrecipes.com/recipe/78144/garlic-chicken-fried-brown-rice/
https://www.allrecipes.com/recipe/16862/thai-chicken/
https://www.allrecipes.com/recipe/76980/fragrant-chicken-curry/
https://www.allrecipes.com/recipe/58097/amazing-simple-thai-tofu/
https://www.allrecipes.com/recipe/21442/lemon-ginger-shrimp/
https://www.allrecipes.com/recipe/34668/chinese-fried-noodles/
https://www.allrecipes.com/recipe/54600/hong-kong-style-egg-tarts/
https://www.allrecipes.com/recipe/88438/teriyaki-chicken-wings/
https://www.allrecipes.com/recipe/33247/absolutely-amazing-ahi/
https://www.allrecipes.com/recipe/7011/chinese-steamed-buns/
https://www.allrecipes.com/recipe/18792/szechuan-spicy-eggplant/
https://www.allrecipes.com/recipe/18252/beef-egg-rolls/
https://www.allrecipes.com/recipe/17137/oriental-dipping-sauce/
https://www.allrecipes.com/recipe/22916/broccoli-and-rice-stir-fry/
https://www.allrecipes.com/recipe/24782/makhani-chicken-indian-butter-chicken/
https://www.allrecipes.com/reci

https://www.allrecipes.com/recipe/84142/stir-fried-shrimp-with-snow-peas-and-ginger/
https://www.allrecipes.com/recipe/33252/indian-style-sheekh-kabab/
https://www.allrecipes.com/recipe/139339/thai-ground-chicken-basil/
https://www.allrecipes.com/recipe/170566/tasty-sesame-tilapia/
https://www.allrecipes.com/recipe/238844/korean-fried-chicken/
https://www.allrecipes.com/recipe/215795/chinese-style-baby-bok-choy-with-mushroom-sauce/
https://www.allrecipes.com/recipe/40351/lumpia-shanghai-version/
https://www.allrecipes.com/recipe/25469/asian-barbequed-steak/
https://www.allrecipes.com/recipe/105691/indian-style-rice-with-cashews-raisins-and-turmeric/
https://www.allrecipes.com/recipe/85498/curried-mustard-greens-with-kidney-beans/
https://www.allrecipes.com/recipe/14680/thit-bo-xao-dau/
19
https://www.allrecipes.com/recipe/19834/kulfi/
https://www.allrecipes.com/recipe/17870/chinese-roast-chicken-with-gravy/
https://www.allrecipes.com/recipe/220181/kung-wow-chicken/
https://www.allrecip

https://www.allrecipes.com/recipe/16474/owens-veggie-stir-fry/
https://www.allrecipes.com/recipe/17626/sweet-and-sour-pork-i/
https://www.allrecipes.com/recipe/13145/mulligatawny-soup-ii/
https://www.allrecipes.com/recipe/162036/korean-spicy-marinated-pork-dae-ji-bool-gogi/
https://www.allrecipes.com/recipe/234642/thai-beef/
https://www.allrecipes.com/recipe/6843/chinkys-mango-bread/
https://www.allrecipes.com/recipe/31400/chi-chi-dango-mochi/
https://www.allrecipes.com/recipe/59280/saffron-rice/
25
https://www.allrecipes.com/recipe/77768/vietnamese-iced-coffee/
https://www.allrecipes.com/recipe/107366/my-chicken-pho-recipe/
https://www.allrecipes.com/recipe/237807/crispy-ginger-beef/
https://www.allrecipes.com/recipe/49350/chinese-green-bean-stir-fry/
https://www.allrecipes.com/recipe/20350/moong-dal/
https://www.allrecipes.com/recipe/22816/chicken-adobo-ii/
https://www.allrecipes.com/recipe/25338/agedashi-esque-tofu/
https://www.allrecipes.com/recipe/13393/long-soup/
https://www.allr

https://www.allrecipes.com/recipe/45381/vegetable-lovers-fried-rice/
31
https://www.allrecipes.com/recipe/24506/shrimp-egg-foo-yung/
https://www.allrecipes.com/recipe/24231/vietnamese-salad-rolls/
https://www.allrecipes.com/recipe/21003/indian-vegetable-rice/
https://www.allrecipes.com/recipe/21182/teriyaki-wraps/
https://www.allrecipes.com/recipe/213146/honey-milk-tea-hong-kong-style/
https://www.allrecipes.com/recipe/236294/haldi-ka-doodh-hot-turmeric-milk/
https://www.allrecipes.com/recipe/138127/rosys-palak-paneer/
https://www.allrecipes.com/recipe/128750/chinese-broccoli/
https://www.allrecipes.com/recipe/161601/curried-cauliflower-soup/
https://www.allrecipes.com/recipe/208039/tandoori-fish/
https://www.allrecipes.com/recipe/212884/channa-masala-chickpea-curry/
https://www.allrecipes.com/recipe/223101/easy-shrimp-lo-mein/
https://www.allrecipes.com/recipe/215055/bengali-chicken-curry-with-potatoes/
https://www.allrecipes.com/recipe/21230/indonesian-pork-satay/
https://www.allreci

https://www.allrecipes.com/recipe/18902/quick-fried-rice/
https://www.allrecipes.com/recipe/17418/shrimp-fried-rice-i/
https://www.allrecipes.com/recipe/87910/spicy-chicken-and-cilantro-wontons/
https://www.allrecipes.com/recipe/79928/easy-raspberry-chicken-with-coconut-rice/
https://www.allrecipes.com/recipe/51542/egg-foo-yung-with-mushroom-sauce/
https://www.allrecipes.com/recipe/220399/half-time-hoisin-chicken-wings/
https://www.allrecipes.com/recipe/220560/kashmiri-lamb/
https://www.allrecipes.com/recipe/213269/easy-spicy-thai-slow-cooker-chicken/
https://www.allrecipes.com/recipe/228240/bibimbap-korean-rice-with-mixed-vegetables/
https://www.allrecipes.com/recipe/148563/hot-and-sour-chicken-and-cabbage-soup/
https://www.allrecipes.com/recipe/140920/broccoli-and-tofu-stir-fry/
https://www.allrecipes.com/recipe/141367/strawberry-lassi/
https://www.allrecipes.com/recipe/136456/vegetable-biryani/
https://www.allrecipes.com/recipe/77437/japanese-potsticker-dip/
https://www.allrecipes.c

https://www.allrecipes.com/recipe/18487/creamy-oriental-dressing/
https://www.allrecipes.com/recipe/9002/sesame-oil-chicken-wings/
https://www.allrecipes.com/recipe/38809/chicken-korma/
https://www.allrecipes.com/recipe/42653/pork-with-peaches-stir-fry/
https://www.allrecipes.com/recipe/53329/braised-tofu/
https://www.allrecipes.com/recipe/47104/pisang-goreng-indonesian-banana-fritters/
https://www.allrecipes.com/recipe/69421/hot-and-sour-tofu-soup-suan-la-dofu-tang/
https://www.allrecipes.com/recipe/212941/cassava-cake/
https://www.allrecipes.com/recipe/212956/empanada-dough/
https://www.allrecipes.com/recipe/198182/halibut-cheeks-with-ginger-orange-sauce/
https://www.allrecipes.com/recipe/204933/okra-curry/
https://www.allrecipes.com/recipe/142636/vietnamese-golden-chicken-wings/
https://www.allrecipes.com/recipe/169856/cream-cheese-and-crab-sushi-rolls/
44
https://www.allrecipes.com/recipe/164254/chinese-mabo-tofu/
https://www.allrecipes.com/recipe/160234/chinese-pepper-round-steak/

https://www.allrecipes.com/recipe/229293/korean-saewoo-bokkeumbap-shrimp-fried-rice/
https://www.allrecipes.com/recipe/228238/goat-stew/
https://www.allrecipes.com/recipe/228067/spicy-asian-cucumbers/
https://www.allrecipes.com/recipe/222697/asian-american-slaw-with-peanuts-and-jalapenos/
https://www.allrecipes.com/recipe/199091/one-egg-egg-drop-soup/
https://www.allrecipes.com/recipe/204890/chinese-five-spice-spare-ribs/
https://www.allrecipes.com/recipe/193868/delish-coconut-muffins/
https://www.allrecipes.com/recipe/136518/asian-ginger-grill-marinade/
https://www.allrecipes.com/recipe/143462/shrimp-summer-rolls-with-asian-peanut-sauce/
https://www.allrecipes.com/recipe/143388/carrot-coconut-lime-soup/
https://www.allrecipes.com/recipe/85508/jemput-jumput-banana-fritters/
https://www.allrecipes.com/recipe/86222/braised-green-beans-with-fried-tofu/
https://www.allrecipes.com/recipe/115038/filipino-pork-adobo/
https://www.allrecipes.com/recipe/47121/marshmallow-cake/


# Concatenating

In [255]:
frames = [df, df1]
result = pd.concat(frames)
result = result.dropna()
result.drop_duplicates()

#Guardamos todos los datos en el csv
data = result.to_csv('recipes.csv', index=False)

In [256]:
result.head()

,Meal,Category,Area,Instructions,MealThumb,Source,Ingredients
0,Garides Saganaki,Seafood,Greek,Place the prawns in a pot and add enough water...,https://www.themealdb.com/images/media/meals/w...,,"500g Raw king prawns, 3 tablespoons Olive oil,..."
1,Chicken Enchilada Casserole,Chicken,Mexican,"Cut each chicken breast in about 3 pieces, so ...",https://www.themealdb.com/images/media/meals/q...,,"14 oz jar Enchilada sauce, 3 Cups shredded Mon..."
2,Bakewell tart,Desert,British,"To make the pastry, measure the flour into a b...",https://www.themealdb.com/images/media/meals/w...,,"175g/6oz plain flour, 75g/2½oz chilled butter,..."
3,Apple Frangipan Tart,Desert,British,Preheat the oven to 200C/180C Fan/Gas 6.\r\nPu...,https://www.themealdb.com/images/media/meals/w...,,"175g/6oz digestive biscuits, 75g/3oz butter, 2..."
4,Kapsalon,Lamb,Dutch,Cut the meat into strips. Heat oil in a pan an...,https://www.themealdb.com/images/media/meals/s...,,"250 Grams Fries, 500 Grams Doner Meat, Topping..."
